# Analysis on Museum Visitor data

In [2]:
from os import environ as ENV
from psycopg2 import connect
from psycopg2.extensions import connection
from psycopg2.extras import RealDictCursor
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
conn = connect(
    host=ENV['DB_HOST'],
    port=ENV['DB_PORT'],
    user=ENV['DB_USER'],
    password=ENV['DB_PASS'],
    database=ENV['DB_NAME'], 
    cursor_factory=RealDictCursor
)

### 1. What exhibition is most frequently visited?

In [5]:
with conn.cursor() as cur:
    query = """
        SELECT 
            e.public_id Exhibition,
            e.exhibition_name,
            count(exhibition_id) 
        FROM rating_interaction
        join exhibition e using(exhibition_id)
        GROUP BY public_id, e.exhibition_name
        ORDER BY count DESC
        LIMIT 1;
    """
    cur.execute(query)
    most_visited = cur.fetchone()
print(f"Exhibition: {most_visited['exhibition']}, Name: {most_visited['exhibition_name']}, Count: {most_visited['count']}")

Exhibition: EXH_04, Name: Our Polluted World, Count: 481


More people interact with the kiosks if more people visit the exhibits, thus the largest number of ratings coincides with the most frequently visited exhibition.

### 2. What hour of the day has the most ratings?

In [6]:
with conn.cursor() as cur:
    query = """
        SELECT
            EXTRACT(HOUR FROM event_at) hour_of_day
        FROM rating_interaction
        GROUP BY hour_of_day
        ORDER BY hour_of_day DESC
        LIMIT 1;
    """
    cur.execute(query)
    most_active = cur.fetchone()
print(f"Hour of day: {most_active['hour_of_day']}")

Hour of day: 18


The hour of the day with the most ratings across all exhibitions is from 6pm GMT, shown above in the 24-hour format.

### 3. What exhibition has the most emergencies?

In [7]:
with conn.cursor() as cur:
    query = """
        SELECT
            e.exhibition_name,
            e.public_id,
            count(r.request_value)
        FROM request_interaction
        JOIN request r USING(request_id)
        JOIN exhibition e USING(exhibition_id)
        WHERE request_value = 1
        GROUP BY e.exhibition_name, e.public_id
        ORDER BY count DESC
        LIMIT 1;
    """
    cur.execute(query)
    most_emergent = cur.fetchone()
print(f"Name: {most_emergent['exhibition_name']}, ID: {most_emergent['public_id']}, Count = {most_emergent['count']}")

Name: The Crenshaw Collection, ID: EXH_02, Count = 2


The Crenshaw Collection ('EXH_002') is the exhibit with the most amount of emergencies, and also the only exhibit with any emergencies.

### 4. What is the average rating for each exhibition?

In [10]:
with conn.cursor() as cur:
    query = """
        SELECT
            e.exhibition_name,
            e.public_id,
            ROUND(AVG(r.rating_value), 2) avg_rating
        FROM rating_interaction
        JOIN rating r USING(rating_id)
        JOIN exhibition e USING(exhibition_id)
        GROUP BY e.public_id, e.exhibition_name
        ORDER BY avg_rating DESC
        ;
    """
    cur.execute(query)
    avg_ratings = cur.fetchall()
for row in avg_ratings:
    print(f"Name: {row['exhibition_name']}, ID: {row['public_id']}, Average Rating: {row['avg_rating']}")

Name: Cetacean Sensations, ID: EXH_03, Average Rating: 2.83
Name: Adaptation, ID: EXH_01, Average Rating: 1.93
Name: Measureless to Man, ID: EXH_00, Average Rating: 1.92
Name: The Crenshaw Collection, ID: EXH_02, Average Rating: 1.43
Name: Our Polluted World, ID: EXH_04, Average Rating: 1.22


Considering the mappings of the rating values (0-4) to the descriptions (Terrible - Amazing), the Cetacean Sensations also had the highest average rating at 2.83. This is on the upper side of Neutral, but closer to an average rating description of Good. The lowest average rating was for Our Polluted World, at a measly 1.22/4, which is closer to Bad than Neutral.

### 5. Are positive ratings more frequent before or after 1pm?

In [11]:
with conn.cursor() as cur:
    query = """
        SELECT
            CASE
                WHEN EXTRACT(HOUR FROM ri.event_at) < 13 THEN 'Before 1pm'
                ELSE 'After 1pm'
            END time_of_day,
            COUNT(CASE WHEN r.rating_value >= 3 THEN 1 END) AS positive_ratings_count
        FROM rating_interaction ri
        JOIN rating r USING(rating_id)
        GROUP BY time_of_day
        ORDER BY time_of_day
        ;
    """
    cur.execute(query)
    positive_ratings = cur.fetchall()
for row in positive_ratings:
    time_of_day = row['time_of_day']
    pos_count = row['positive_ratings_count']
    print(f"{time_of_day}: {pos_count} positive ratings")

After 1pm: 344 positive ratings
Before 1pm: 313 positive ratings


Positive ratings were classified as having a value equal or greater than 3, which directly corresponds to the description of Good or Amazing. The data shows that more positive ratings were received after 1pm, however there is a fine margin of only about 10% of the amount before 1pm. 

### 6. Do Zoology exhibitions get better ratings than other types?

In [13]:
with conn.cursor() as cur:
    query = """
        SELECT
            CASE
                WHEN d.department_name = 'Zoology' THEN 'Zoology'
                ELSE 'Other Departments'
            END department,
            ROUND(AVG(r.rating_value), 2) avg_rating
        FROM rating_interaction ri
        JOIN rating r USING(rating_id)
        JOIN exhibition e USING(exhibition_id)
        JOIN department d USING(department_id)
        GROUP BY department
        ORDER BY department DESC
        ;
    """
    cur.execute(query)
    dep_avg_rating_zoo = cur.fetchall()
for row in dep_avg_rating_zoo:
    dep = row['department']
    print(f"{dep}: {row['avg_rating']} average rating")

Zoology: 2.20 average rating
Other Departments: 1.64 average rating


The above data shows that the Zoology department's exhibitions actually get a higher average rating than other departments. With 2.2/4, the average rating for Zoology is between Neutral and Good, whereas other department's average rating falls below the Neutral category at 1.64/4.

In [22]:
conn.close()